# Train the Model
We'll start to train model with preprocessed data wich was separated to train the model dedicatedly. For which we'll start with  Splitting into feature and target columns and then we'll select features which affects the target columns more than others and decide about what feature should keep to train and what should not.



importing necessary libraries

In [1]:
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import randint

In [2]:
data = pd.read_csv('../../data/processed/train_validation_data_scaled.csv')
data

,Age,Gender,EducationBackground,MaritalStatus,EmpDepartment,EmpJobRole,BusinessTravelFrequency,DistanceFromHome,EmpEducationLevel,EmpEnvironmentSatisfaction,...,EmpRelationshipSatisfaction,TotalWorkExperienceInYears,TrainingTimesLastYear,EmpWorkLifeBalance,ExperienceYearsAtThisCompany,ExperienceYearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,PerformanceRating
0,0.523810,1.0,0.2,0.5,1.0,0.722222,0.5,0.142857,0.75,1.000000,...,0.666667,0.500,0.333333,0.666667,0.450,0.722222,0.066667,0.705882,0.0,4
1,0.214286,0.0,0.2,1.0,0.8,0.500000,0.5,0.142857,0.25,1.000000,...,0.666667,0.150,0.500000,0.333333,0.125,0.222222,0.000000,0.117647,0.0,3
2,0.333333,0.0,0.6,1.0,0.8,0.888889,0.0,1.000000,0.75,1.000000,...,0.333333,0.250,0.333333,0.666667,0.200,0.388889,0.466667,0.411765,0.0,3
3,0.619048,1.0,0.6,1.0,0.8,0.666667,1.0,0.000000,0.50,0.333333,...,0.666667,0.150,0.333333,0.666667,0.100,0.166667,0.066667,0.117647,0.0,2
4,0.833333,0.0,1.0,0.5,0.8,0.555556,1.0,0.035714,1.00,0.666667,...,0.333333,0.850,0.666667,0.666667,0.825,0.388889,0.066667,0.529412,1.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,0.166667,1.0,1.0,0.5,0.8,0.388889,1.0,0.000000,0.50,0.000000,...,0.333333,0.175,0.333333,0.333333,0.075,0.111111,0.000000,0.117647,0.0,3
956,0.214286,0.0,0.0,0.5,0.6,0.333333,0.5,0.750000,0.50,0.000000,...,0.000000,0.025,0.333333,0.666667,0.025,0.000000,0.000000,0.000000,1.0,3
957,0.571429,1.0,0.2,0.0,0.8,0.666667,1.0,0.607143,0.75,1.000000,...,0.666667,0.225,0.500000,0.333333,0.100,0.166667,0.066667,0.117647,0.0,3
958,0.166667,1.0,0.6,0.5,0.2,0.000000,1.0,0.214286,0.00,1.000000,...,0.666667,0.050,0.333333,0.666667,0.050,0.111111,0.133333,0.058824,0.0,3


### Split feature and target columns

In [3]:
X = data.drop('PerformanceRating', axis=1)
y = data['PerformanceRating']

### Feature Selection: Chi Square Test

- Chi square test will check the association between categorical variables 

- If the association with the target is low, the variables will be dropped.

In [4]:
bestfeatures = SelectKBest(score_func=chi2,k='all')
fit = bestfeatures.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featurescores = pd.concat([dfcolumns,dfscores],axis=1)
featurescores.columns = ['feature_names','Scores']

print(featurescores.nlargest(27,'Scores'))

                   feature_names     Scores
16      EmpLastSalaryHikePercent  67.766322
9     EmpEnvironmentSatisfaction  46.116146
23       YearsSinceLastPromotion  12.191408
15                      OverTime   7.756564
4                  EmpDepartment   6.955023
22  ExperienceYearsInCurrentRole   4.434895
24          YearsWithCurrManager   4.119061
21  ExperienceYearsAtThisCompany   2.584402
25                     Attrition   2.067551
12                   EmpJobLevel   1.814980
5                     EmpJobRole   1.771901
20            EmpWorkLifeBalance   1.623742
1                         Gender   0.938957
18    TotalWorkExperienceInYears   0.583482
10                 EmpHourlyRate   0.582173
3                  MaritalStatus   0.478578
8              EmpEducationLevel   0.375576
6        BusinessTravelFrequency   0.307819
7               DistanceFromHome   0.226600
19         TrainingTimesLastYear   0.217953
0                            Age   0.124409
13            EmpJobSatisfaction

#### Columns that drops

The columns are **selected** for training as features which **Chi square test score is above 1**. Accrording to this selected score the columns which falls below 1 and should drops are:  EmpEducationLevel, MaritalStatus, Gender, BusinessTravelFrequency, EmpJobSatisfaction, EmpJobInvolvement, EmpRelationshipSatisfaction, NumCompaniesWorked, EducationBackground, Age, TrainingTimesLastYear, DistanceFromHome.

In [5]:
X

,Age,Gender,EducationBackground,MaritalStatus,EmpDepartment,EmpJobRole,BusinessTravelFrequency,DistanceFromHome,EmpEducationLevel,EmpEnvironmentSatisfaction,...,EmpLastSalaryHikePercent,EmpRelationshipSatisfaction,TotalWorkExperienceInYears,TrainingTimesLastYear,EmpWorkLifeBalance,ExperienceYearsAtThisCompany,ExperienceYearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0.523810,1.0,0.2,0.5,1.0,0.722222,0.5,0.142857,0.75,1.000000,...,0.714286,0.666667,0.500,0.333333,0.666667,0.450,0.722222,0.066667,0.705882,0.0
1,0.214286,0.0,0.2,1.0,0.8,0.500000,0.5,0.142857,0.25,1.000000,...,0.214286,0.666667,0.150,0.500000,0.333333,0.125,0.222222,0.000000,0.117647,0.0
2,0.333333,0.0,0.6,1.0,0.8,0.888889,0.0,1.000000,0.75,1.000000,...,0.071429,0.333333,0.250,0.333333,0.666667,0.200,0.388889,0.466667,0.411765,0.0
3,0.619048,1.0,0.6,1.0,0.8,0.666667,1.0,0.000000,0.50,0.333333,...,0.071429,0.666667,0.150,0.333333,0.666667,0.100,0.166667,0.066667,0.117647,0.0
4,0.833333,0.0,1.0,0.5,0.8,0.555556,1.0,0.035714,1.00,0.666667,...,0.357143,0.333333,0.850,0.666667,0.666667,0.825,0.388889,0.066667,0.529412,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,0.166667,1.0,1.0,0.5,0.8,0.388889,1.0,0.000000,0.50,0.000000,...,0.000000,0.333333,0.175,0.333333,0.333333,0.075,0.111111,0.000000,0.117647,0.0
956,0.214286,0.0,0.0,0.5,0.6,0.333333,0.5,0.750000,0.50,0.000000,...,0.071429,0.000000,0.025,0.333333,0.666667,0.025,0.000000,0.000000,0.000000,1.0
957,0.571429,1.0,0.2,0.0,0.8,0.666667,1.0,0.607143,0.75,1.000000,...,0.428571,0.666667,0.225,0.500000,0.333333,0.100,0.166667,0.066667,0.117647,0.0
958,0.166667,1.0,0.6,0.5,0.2,0.000000,1.0,0.214286,0.00,1.000000,...,0.000000,0.666667,0.050,0.333333,0.666667,0.050,0.111111,0.133333,0.058824,0.0


In [6]:
X = X.drop(['EmpEducationLevel', 'MaritalStatus', 'Gender', 'BusinessTravelFrequency', 'EmpJobSatisfaction', 'EmpJobInvolvement', 'EmpRelationshipSatisfaction', 'NumCompaniesWorked', 'EducationBackground', 'Age', 'TrainingTimesLastYear','DistanceFromHome'], axis=1)
X

,EmpDepartment,EmpJobRole,EmpEnvironmentSatisfaction,EmpHourlyRate,EmpJobLevel,OverTime,EmpLastSalaryHikePercent,TotalWorkExperienceInYears,EmpWorkLifeBalance,ExperienceYearsAtThisCompany,ExperienceYearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,1.0,0.722222,1.000000,0.257143,0.50,1.0,0.714286,0.500,0.666667,0.450,0.722222,0.066667,0.705882,0.0
1,0.8,0.500000,1.000000,0.328571,0.00,1.0,0.214286,0.150,0.333333,0.125,0.222222,0.000000,0.117647,0.0
2,0.8,0.888889,1.000000,0.557143,0.00,0.0,0.071429,0.250,0.666667,0.200,0.388889,0.466667,0.411765,0.0
3,0.8,0.666667,0.333333,0.171429,0.00,1.0,0.071429,0.150,0.666667,0.100,0.166667,0.066667,0.117647,0.0
4,0.8,0.555556,0.666667,0.685714,0.50,0.0,0.357143,0.850,0.666667,0.825,0.388889,0.066667,0.529412,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,0.8,0.388889,0.000000,0.728571,0.00,0.0,0.000000,0.175,0.333333,0.075,0.111111,0.000000,0.117647,0.0
956,0.6,0.333333,0.000000,0.400000,0.00,0.0,0.071429,0.025,0.666667,0.025,0.000000,0.000000,0.000000,1.0
957,0.8,0.666667,1.000000,0.071429,0.25,1.0,0.428571,0.225,0.333333,0.100,0.166667,0.066667,0.117647,0.0
958,0.2,0.000000,1.000000,0.485714,0.00,0.0,0.000000,0.050,0.666667,0.050,0.111111,0.133333,0.058824,0.0


In [7]:
X.shape, y.shape

((960, 14), (960,))

### Splitting the Training and Testing Data

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X,y, random_state=42, test_size=0.20)

In [9]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((768, 14), (192, 14), (768,), (192,))

# Model Creation

### **Logistic Regression**

In [10]:
logistic = LogisticRegression(random_state=42)
logistic.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [11]:
logistic_val_pred = logistic.predict(X_val)
logistic_train_pred = logistic.predict(X_train)

In [12]:
accuracy_val_logistic = accuracy_score(y_val, logistic_val_pred)
accuracy_train_logistic = accuracy_score(y_train, logistic_train_pred)

print(f"Validation Accuracy: {accuracy_val_logistic}")
print(f"Train Accuracy: {accuracy_train_logistic}")

Validation Accuracy: 0.8333333333333334
Train Accuracy: 0.83203125


In [13]:
print(classification_report(y_val, logistic_val_pred))

              precision    recall  f1-score   support

           2       0.57      0.43      0.49        28
           3       0.86      0.94      0.90       149
           4       0.89      0.53      0.67        15

    accuracy                           0.83       192
   macro avg       0.77      0.63      0.69       192
weighted avg       0.82      0.83      0.82       192



### **Naive Bayes**

In [14]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [15]:
gnb_val_pred = gnb.predict(X_val)
gnb_train_pred = gnb.predict(X_train)

In [16]:
accuracy_val_gnb = accuracy_score(y_val, gnb_val_pred)
accuracy_train_gnb = accuracy_score(y_train, gnb_train_pred)

print(f"Validation Accuracy: {accuracy_val_gnb}")
print(f"Train Accuracy: {accuracy_train_gnb}")

Validation Accuracy: 0.828125
Train Accuracy: 0.8125


In [17]:
print(classification_report(y_val, gnb_val_pred))

              precision    recall  f1-score   support

           2       0.54      0.50      0.52        28
           3       0.88      0.90      0.89       149
           4       0.79      0.73      0.76        15

    accuracy                           0.83       192
   macro avg       0.74      0.71      0.72       192
weighted avg       0.82      0.83      0.83       192



### **KNN Classifier**

In [18]:
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean')

In [19]:
knn_val_pred = knn.predict(X_val)
knn_train_pred = knn.predict(X_train)

In [20]:
accuracy_val_knn = accuracy_score(y_val, knn_val_pred)
accuracy_train_knn = accuracy_score(y_train, knn_train_pred)

print(f"Validation Accuracy: {accuracy_val_knn}")
print(f"Train Accuracy: {accuracy_train_knn}")

Validation Accuracy: 0.828125
Train Accuracy: 0.8463541666666666


In [21]:
print(classification_report(y_val, knn_val_pred))

              precision    recall  f1-score   support

           2       0.71      0.61      0.65        28
           3       0.86      0.93      0.89       149
           4       0.50      0.27      0.35        15

    accuracy                           0.83       192
   macro avg       0.69      0.60      0.63       192
weighted avg       0.81      0.83      0.82       192



### **SVM Classifier**

In [22]:
svm_clf = svm.SVC(kernel='rbf', C = 10, gamma = 0.1, random_state=42)
svm_clf.fit(X_train, y_train)

SVC(C=10, gamma=0.1, random_state=42)

In [23]:
svm_val_pred = svm_clf.predict(X_val)
svm_train_pred = svm_clf.predict(X_train)

In [24]:
accuracy_val_svm = accuracy_score(y_val, svm_val_pred)
accuracy_train_svm = accuracy_score(y_train, svm_train_pred)

print(f"Validation Accuracy: {accuracy_val_svm}")
print(f"Train Accuracy: {accuracy_train_svm}")

Validation Accuracy: 0.8489583333333334
Train Accuracy: 0.8658854166666666


In [25]:
print(classification_report(y_val, svm_val_pred))

              precision    recall  f1-score   support

           2       0.65      0.54      0.59        28
           3       0.88      0.93      0.91       149
           4       0.82      0.60      0.69        15

    accuracy                           0.85       192
   macro avg       0.78      0.69      0.73       192
weighted avg       0.84      0.85      0.84       192



### **Decision Tree Classifier**

In [26]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [27]:
dt_val_pred = dt.predict(X_val)
dt_train_pred = dt.predict(X_train)

In [28]:
accuracy_val_dt = accuracy_score(y_val, dt_val_pred)
accuracy_train_dt = accuracy_score(y_train, dt_train_pred)

print(f"Validation Accuracy: {accuracy_val_dt}")
print(f"Train Accuracy: {accuracy_train_dt}")

Validation Accuracy: 0.8958333333333334
Train Accuracy: 1.0


In [29]:
print(classification_report(y_val, dt_val_pred))

              precision    recall  f1-score   support

           2       0.73      0.79      0.76        28
           3       0.95      0.92      0.94       149
           4       0.72      0.87      0.79        15

    accuracy                           0.90       192
   macro avg       0.80      0.86      0.83       192
weighted avg       0.90      0.90      0.90       192



### **Random Forest Classifier**

In [30]:
rfc=RandomForestClassifier(bootstrap= True,max_depth= 7,max_features= 15,min_samples_leaf= 3,min_samples_split= 10,
                           n_estimators= 200,
                           random_state=42)
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=7, max_features=15, min_samples_leaf=3,
                       min_samples_split=10, n_estimators=200, random_state=42)

In [31]:
rfc_val_pred = rfc.predict(X_val)
rfc_train_pred = rfc.predict(X_train)

In [32]:
accuracy_val_rfc = accuracy_score(y_val, rfc_val_pred)
accuracy_train_rfc = accuracy_score(y_train, rfc_train_pred)

print(f"Validation Accuracy: {accuracy_val_rfc}")
print(f"Train Accuracy: {accuracy_train_rfc}")

Validation Accuracy: 0.9322916666666666
Train Accuracy: 0.9544270833333334


In [33]:
print(classification_report(y_val, rfc_val_pred))

              precision    recall  f1-score   support

           2       0.77      0.86      0.81        28
           3       0.96      0.96      0.96       149
           4       1.00      0.80      0.89        15

    accuracy                           0.93       192
   macro avg       0.91      0.87      0.89       192
weighted avg       0.94      0.93      0.93       192



### **Gradient Boosting Classifier**

In [34]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbc.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

In [35]:
gbc_val_pred = gbc.predict(X_val)
gbc_train_pred = gbc.predict(X_train)

In [36]:
accuracy_val_gbc = accuracy_score(y_val, gbc_val_pred)
accuracy_train_gbc = accuracy_score(y_train, gbc_train_pred)

print(f"Validation Accuracy: {accuracy_val_gbc}")
print(f"Train Accuracy: {accuracy_train_gbc}")

Validation Accuracy: 0.9270833333333334
Train Accuracy: 0.9934895833333334


## **Use Hyperparameter Tuning**

- Hyperparameter Tunning is the process of finding the best set of hyperparameters for a machine learning model to achieve optimal performance on a given dataset.
- In hyperparameter tunning the model perform and calculate accuracy with all possible parameter given to it internally and give the best parameters which give the best performance.
- **Grid Search and Random Search are two commonly used methods for Hyperparameter tunning**

### **Logistic Regression**

#### GridSearchCV

In [37]:
param_grid = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'penalty': ['l1', 'l2', 'elasticent'],
    'C': [0.001, 0.01, 0.1, 1, 10],
    'multi_class':['ovr','multinomial']
}

logistic = LogisticRegression(random_state=42)
grid_search = GridSearchCV(logistic, param_grid, cv = 5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(best_params)

{'C': 10, 'multi_class': 'multinomial', 'penalty': 'l1', 'solver': 'saga'}


In [38]:
logistic_gscv = LogisticRegression(**best_params, random_state=42)
logistic_gscv.fit(X_train, y_train)

logistic_gscv_val_pred = logistic_gscv.predict(X_val)
logistic_gscv_train_pred = logistic_gscv.predict(X_train)

accuracy_val_logistic_gscv = accuracy_score(y_val, logistic_gscv_val_pred)
accuracy_train_logistic_gscv = accuracy_score(y_train, logistic_gscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_logistic_gscv}")
print(f"Train Accuracy: {accuracy_train_logistic_gscv}")

Validation Accuracy: 0.8385416666666666
Train Accuracy: 0.8372395833333334


#### RandomizedSearchCV

In [39]:
param_dist = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'penalty': ['l1', 'l2', 'elasticent'],
    'multi_class':['ovr','multinomial'],
    'C': uniform(loc=0, scale=10)
}

logistic = LogisticRegression(random_state=42)
random_search = RandomizedSearchCV(logistic, param_distributions=param_dist, cv = 5)
random_search.fit(X_train, y_train)
best_params = random_search.best_params_
print(best_params)

{'C': 8.384346333322712, 'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'saga'}


In [40]:
logistic_rscv = LogisticRegression(**best_params, random_state=42)
logistic_rscv.fit(X_train, y_train)

logistic_rscv_val_pred = logistic_rscv.predict(X_val)
logistic_rscv_train_pred = logistic_rscv.predict(X_train)

accuracy_val_logistic_rscv = accuracy_score(y_val, logistic_rscv_val_pred)
accuracy_train_logistic_rscv = accuracy_score(y_train, logistic_rscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_logistic_rscv}")
print(f"Train Accuracy: {accuracy_train_logistic_rscv}")

Validation Accuracy: 0.8333333333333334
Train Accuracy: 0.8385416666666666


In [72]:
# Dictionary to hold accuracy values and corresponding models
accuracy_models_logistic = {
    accuracy_val_logistic_gscv: logistic_gscv,
    accuracy_val_logistic_rscv: logistic_rscv
}

# Get the maximum accuracy and the corresponding model variable name
max_accuracy_logistic = max(accuracy_models_logistic)
best_accuracy_logistic_model = accuracy_models_logistic[max_accuracy_logistic]

# Print the maximum accuracy and the corresponding model variable name
print(f"Max accuracy: {max_accuracy_logistic} (from {best_accuracy_logistic_model})")

# Store the variable name of the best model
%store best_accuracy_logistic_model
%store max_accuracy_logistic

Max accuracy: 0.8385416666666666 (from LogisticRegression(C=10, multi_class='multinomial', penalty='l1',
                   random_state=42, solver='saga'))
Stored 'best_accuracy_logistic_model' (LogisticRegression)
Stored 'max_accuracy_logistic' (float64)


### **Naive Bayes**
#### GridSearchCV

In [42]:
param_grid = {
    'alpha': [0.1, 1, 10],
    'fit_prior': [True, False]
}

nb = MultinomialNB()
grid_search = GridSearchCV(nb, param_grid, cv = 5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(best_params)

{'alpha': 0.1, 'fit_prior': True}


In [43]:
nb_gscv = MultinomialNB(**best_params)
nb_gscv.fit(X_train, y_train)

nb_gscv_val_pred = nb_gscv.predict(X_val)
nb_gscv_train_pred = nb_gscv.predict(X_train)

accuracy_val_nb_gscv = accuracy_score(y_val, nb_gscv_val_pred)
accuracy_train_nb_gscv = accuracy_score(y_train, nb_gscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_nb_gscv}")
print(f"Train Accuracy: {accuracy_train_nb_gscv}")

Validation Accuracy: 0.7760416666666666
Train Accuracy: 0.7122395833333334


#### RandomizedSearchCV

In [44]:
param_dist = {
    'alpha': uniform(loc=0, scale=10)
}

nb = MultinomialNB()
random_search = RandomizedSearchCV(nb, param_distributions= param_dist, cv = 5)
random_search.fit(X_train, y_train)
best_params = random_search.best_params_
print(best_params)

{'alpha': 3.9334325246690116}


In [45]:
nb_rscv = MultinomialNB(**best_params)
nb_rscv.fit(X_train, y_train)

nb_rscv_val_pred = nb_rscv.predict(X_val)
nb_rscv_train_pred = nb_rscv.predict(X_train)

accuracy_val_nb_rscv = accuracy_score(y_val, nb_rscv_val_pred)
accuracy_train_nb_rscv = accuracy_score(y_train, nb_rscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_nb_rscv}")
print(f"Train Accuracy: {accuracy_train_nb_rscv}")

Validation Accuracy: 0.7760416666666666
Train Accuracy: 0.7122395833333334


In [73]:
# Dictionary to hold accuracy values and corresponding models
accuracy_models_nb = {
    accuracy_val_nb_gscv: nb_gscv,
    accuracy_val_nb_rscv: nb_rscv
}

# Get the maximum accuracy and the corresponding model variable name
max_accuracy_nb = max(accuracy_models_nb)
best_accuracy_nb_model = accuracy_models_nb[max_accuracy_nb]

# Print the maximum accuracy and the corresponding model variable name
print(f"Max accuracy: {max_accuracy_nb} (from {best_accuracy_nb_model})")

# Store the variable name of the best model
%store best_accuracy_nb_model
%store max_accuracy_nb

Max accuracy: 0.7760416666666666 (from MultinomialNB(alpha=3.9334325246690116))
Stored 'best_accuracy_nb_model' (MultinomialNB)
Stored 'max_accuracy_nb' (float64)


### **Decision Tree Classifier**

#### GridSearchCV

In [47]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'splitter': ['best', 'random']
}

dt = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(dt, param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(best_params)

{'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'splitter': 'best'}


In [48]:
dt_gscv = DecisionTreeClassifier(**best_params, random_state=42)
dt_gscv.fit(X_train, y_train)

dt_gscv_val_pred = dt_gscv.predict(X_val)
dt_gscv_train_pred = dt_gscv.predict(X_train)

accuracy_val_dt_gscv = accuracy_score(y_val, dt_gscv_val_pred)
accuracy_train_dt_gscv = accuracy_score(y_train, dt_gscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_dt_gscv}")
print(f"Train Accuracy: {accuracy_train_dt_gscv}")

Validation Accuracy: 0.8489583333333334
Train Accuracy: 0.921875


#### Randomized Search CV

In [49]:
param_dist = {
    'max_depth': randint(1, 20),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['auto', 'sqrt', 'log2']
}

dt = DecisionTreeClassifier(random_state=42)
random_search = RandomizedSearchCV(dt, param_distributions=param_dist, cv=5)
random_search.fit(X_train, y_train)
best_params = random_search.best_params_
print(best_params)

{'max_depth': 14, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 13}


In [50]:
dt_rscv = DecisionTreeClassifier(**best_params, random_state=42)
dt_rscv.fit(X_train, y_train)

dt_rscv_val_pred = dt_rscv.predict(X_val)
dt_rscv_train_pred = dt_rscv.predict(X_train)

accuracy_val_dt_rscv = accuracy_score(y_val, dt_rscv_val_pred)
accuracy_train_dt_rscv = accuracy_score(y_train, dt_rscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_dt_rscv}")
print(f"Train Accuracy: {accuracy_train_dt_rscv}")

Validation Accuracy: 0.828125
Train Accuracy: 0.9127604166666666


In [74]:
# Dictionary to hold accuracy values and corresponding models
accuracy_models_dt = {
    accuracy_val_dt_gscv: dt_gscv,
    accuracy_val_dt_rscv: dt_rscv
}

# Get the maximum accuracy and the corresponding model variable name
max_accuracy_dt = max(accuracy_models_dt)
best_accuracy_dt_model = accuracy_models_dt[max_accuracy_dt]

# Print the maximum accuracy and the corresponding model variable name
print(f"Max accuracy: {max_accuracy_dt} (from {best_accuracy_dt_model})")

# Store the variable name of the best model
%store best_accuracy_dt_model
%store max_accuracy_dt

Max accuracy: 0.8489583333333334 (from DecisionTreeClassifier(max_depth=10, max_features='sqrt', min_samples_split=10,
                       random_state=42))
Stored 'best_accuracy_dt_model' (DecisionTreeClassifier)
Stored 'max_accuracy_dt' (float64)


### **KNN**
#### Grid Search CV

In [52]:
param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'], 
    'p': [1, 2]
}

knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(best_params)

{'metric': 'euclidean', 'n_neighbors': 7, 'p': 1, 'weights': 'distance'}


In [53]:
knn_gscv = KNeighborsClassifier(**best_params)
knn_gscv.fit(X_train, y_train)

knn_gscv_val_pred = knn_gscv.predict(X_val)
knn_gscv_train_pred = knn_gscv.predict(X_train)

accuracy_val_knn_gscv = accuracy_score(y_val, knn_gscv_val_pred)
accuracy_train_knn_gscv = accuracy_score(y_train, knn_gscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_knn_gscv}")
print(f"Train Accuracy: {accuracy_train_knn_gscv}")

Validation Accuracy: 0.8072916666666666
Train Accuracy: 1.0


#### Randomized Search CV

In [54]:
param_dist = {
    'n_neighbors': randint(1, 20),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'p': [1, 2]
}

knn = KNeighborsClassifier()
random_search = RandomizedSearchCV(knn, param_distributions = param_dist, cv=5)
random_search.fit(X_train, y_train)
best_params = random_search.best_params_
print(best_params)

{'metric': 'minkowski', 'n_neighbors': 17, 'p': 1, 'weights': 'distance'}


In [55]:
knn_rscv = KNeighborsClassifier(**best_params)
knn_rscv.fit(X_train, y_train)

knn_rscv_val_pred = knn_rscv.predict(X_val)
knn_rscv_train_pred = knn_rscv.predict(X_train)

accuracy_val_knn_rscv = accuracy_score(y_val, knn_rscv_val_pred)
accuracy_train_knn_rscv = accuracy_score(y_train, knn_rscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_knn_rscv}")
print(f"Train Accuracy: {accuracy_train_knn_rscv}")

Validation Accuracy: 0.8333333333333334
Train Accuracy: 1.0


In [75]:
# Dictionary to hold accuracy values and corresponding models
accuracy_models_knn = {
    accuracy_val_knn_gscv: knn_gscv,
    accuracy_val_knn_rscv: knn_rscv
}

# Get the maximum accuracy and the corresponding model variable name
max_accuracy_knn = max(accuracy_models_knn)
best_accuracy_knn_model = accuracy_models_knn[max_accuracy_knn]

# Print the maximum accuracy and the corresponding model variable name
print(f"Max accuracy: {max_accuracy_knn} (from {best_accuracy_knn_model})")

# Store the variable name of the best model
%store best_accuracy_knn_model
%store max_accuracy_knn

Max accuracy: 0.8333333333333334 (from KNeighborsClassifier(n_neighbors=17, p=1, weights='distance'))
Stored 'best_accuracy_knn_model' (KNeighborsClassifier)
Stored 'max_accuracy_knn' (float64)


### **Random Forest Classifier**
#### Grid Search CV

In [57]:
param_grid = {'n_estimators': [50, 100, 150],
              'max_depth': [None, 5,10,15, 20],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4] }
rfc = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rfc, param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(best_params)

{'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}


In [58]:
rfc_gscv = RandomForestClassifier(**best_params, random_state=42)
rfc_gscv.fit(X_train, y_train)

rfc_gscv_val_pred = rfc_gscv.predict(X_val)
rfc_gscv_train_pred = rfc_gscv.predict(X_train)

accuracy_val_rfc_gscv = accuracy_score(y_val, rfc_gscv_val_pred)
accuracy_train_rfc_gscv = accuracy_score(y_train, rfc_gscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_rfc_gscv}")
print(f"Train Accuracy: {accuracy_train_rfc_gscv}")

Validation Accuracy: 0.9427083333333334
Train Accuracy: 0.9765625


#### Randomized Search CV

In [59]:
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

rfc = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(rfc, param_grid, cv=5)
random_search.fit(X_train, y_train)
best_params = random_search.best_params_
print(best_params)

{'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 20, 'criterion': 'gini'}


In [60]:
rfc_rscv = RandomForestClassifier(**best_params, random_state=42)
rfc_rscv.fit(X_train, y_train)

rfc_rscv_val_pred = rfc_rscv.predict(X_val)
rfc_rscv_train_pred = rfc_rscv.predict(X_train)

accuracy_val_rfc_rscv = accuracy_score(y_val, rfc_rscv_val_pred)
accuracy_train_rfc_rscv = accuracy_score(y_train, rfc_rscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_rfc_rscv}")
print(f"Train Accuracy: {accuracy_train_rfc_rscv}")

Validation Accuracy: 0.9322916666666666
Train Accuracy: 0.9661458333333334


In [76]:
# Dictionary to hold accuracy values and corresponding models
accuracy_models_rfc = {
    accuracy_val_rfc_gscv: rfc_gscv,
    accuracy_val_rfc_rscv: rfc_rscv
}

# Get the maximum accuracy and the corresponding model variable name
max_accuracy_rfc = max(accuracy_models_rfc)
best_accuracy_rfc_model = accuracy_models_rfc[max_accuracy_rfc]

# Print the maximum accuracy and the corresponding model variable name
print(f"Max accuracy: {max_accuracy_rfc} (from {best_accuracy_rfc_model})")

# Store the variable name of the best model
%store best_accuracy_rfc_model
%store max_accuracy_rfc

Max accuracy: 0.9427083333333334 (from RandomForestClassifier(max_depth=15, min_samples_split=5, random_state=42))
Stored 'best_accuracy_rfc_model' (RandomForestClassifier)
Stored 'max_accuracy_rfc' (float64)


### **SVM Classifier**

#### Grid Search CV


In [62]:
param_grid = {'C':[0.1,0.5,10,50,60,70,80],
             'gamma':[1,0.1,0.001,0.0001,0.00001]}

svm_clf = svm.SVC(random_state=42)
grid_search = GridSearchCV(svm_clf, param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(best_params)


{'C': 10, 'gamma': 1}


In [63]:
svm_gscv = svm.SVC(**best_params, random_state=42)
svm_gscv.fit(X_train, y_train)

svm_gscv_val_pred = svm_gscv.predict(X_val)
svm_gscv_train_pred = svm_gscv.predict(X_train)

accuracy_val_svm_gscv = accuracy_score(y_val, svm_gscv_val_pred)
accuracy_train_svm_gscv = accuracy_score(y_train, svm_gscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_svm_gscv}")
print(f"Train Accuracy: {accuracy_train_svm_gscv}")

Validation Accuracy: 0.8489583333333334
Train Accuracy: 0.9830729166666666


#### Randomized Search CV

In [64]:
param_grid = {'C':[0.1,0.5,10,50,60,70,80],
             'gamma':[1,0.1,0.001,0.0001,0.00001]}

svm_clf = svm.SVC(random_state=42)
random_search = RandomizedSearchCV(svm_clf, param_grid, cv=5)
random_search.fit(X_train, y_train)
best_params = random_search.best_params_
print(best_params)

{'gamma': 1, 'C': 10}


In [65]:
svm_rscv = svm.SVC(**best_params, random_state=42)
svm_rscv.fit(X_train, y_train)

svm_rscv_val_pred = svm_rscv.predict(X_val)
svm_rscv_train_pred = svm_rscv.predict(X_train)

accuracy_val_svm_rscv = accuracy_score(y_val, svm_rscv_val_pred)
accuracy_train_svm_rscv = accuracy_score(y_train, svm_rscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_svm_rscv}")
print(f"Train Accuracy: {accuracy_train_svm_rscv}")

Validation Accuracy: 0.8489583333333334
Train Accuracy: 0.9830729166666666


In [77]:
# Dictionary to hold accuracy values and corresponding models
accuracy_models_svm = {
    accuracy_val_svm_gscv: svm_gscv,
    accuracy_val_svm_rscv: svm_rscv
}

# Get the maximum accuracy and the corresponding model variable name
max_accuracy_svm = max(accuracy_models_svm)
best_accuracy_svm_model = accuracy_models_svm[max_accuracy_svm]

# Print the maximum accuracy and the corresponding model variable name
print(f"Max accuracy: {max_accuracy_svm} (from {best_accuracy_svm_model})")

# Store the variable name of the best model
%store best_accuracy_svm_model
%store max_accuracy_svm

Max accuracy: 0.8489583333333334 (from SVC(C=10, gamma=1, random_state=42))
Stored 'best_accuracy_svm_model' (SVC)
Stored 'max_accuracy_svm' (float64)


### **Gradient Boosting Classifier**

#### Grid Search CV

In [67]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
}

gbc = GradientBoostingClassifier(random_state = 42)
grid_search = GridSearchCV(gbc, param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(best_params)

{'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50, 'subsample': 0.9}


In [68]:
gbc_gscv = GradientBoostingClassifier(**best_params, random_state=42)
gbc_gscv.fit(X_train, y_train)

gbc_gscv_val_pred = gbc_gscv.predict(X_val)
gbc_gscv_train_pred = gbc_gscv.predict(X_train)

accuracy_val_gbc_gscv = accuracy_score(y_val, gbc_gscv_val_pred)
accuracy_train_gbc_gscv = accuracy_score(y_train, gbc_gscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_gbc_gscv}")
print(f"Train Accuracy: {accuracy_train_gbc_gscv}")

Validation Accuracy: 0.921875
Train Accuracy: 0.96875


#### Randomized Search CV

In [69]:
param_grid = {
    'n_estimators': randint(1, 200),
    'learning_rate': randint(0, 10),
    'max_depth': randint(1, 20),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'subsample': [0.8, 0.9, 1.0]
}

gbc = GradientBoostingClassifier(random_state = 42)
random_search = RandomizedSearchCV(gbc, param_grid, cv=5)
random_search.fit(X_train, y_train)
best_params = random_search.best_params_
print(best_params)

{'learning_rate': 1, 'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 90, 'subsample': 1.0}


In [70]:
gbc_rscv = GradientBoostingClassifier(**best_params, random_state=42)
gbc_rscv.fit(X_train, y_train)

gbc_rscv_val_pred = gbc_rscv.predict(X_val)
gbc_rscv_train_pred = gbc_rscv.predict(X_train)

accuracy_val_gbc_rscv = accuracy_score(y_val, gbc_rscv_val_pred)
accuracy_train_gbc_rscv = accuracy_score(y_train, gbc_rscv_train_pred)

print(f"Validation Accuracy: {accuracy_val_gbc_rscv}")
print(f"Train Accuracy: {accuracy_train_gbc_rscv}")

Validation Accuracy: 0.921875
Train Accuracy: 1.0


In [83]:
# Dictionary to hold accuracy values and corresponding models
accuracy_models_gbc = {
    accuracy_val_gbc_gscv: gbc_gscv,
    accuracy_val_gbc_rscv: gbc_rscv
}

# Get the maximum accuracy and the corresponding model variable name
max_accuracy_gbc = max(accuracy_models_gbc)
best_accuracy_gbc_model = accuracy_models_gbc[max_accuracy_gbc]

# Print the maximum accuracy and the corresponding model variable name
print(f"Max accuracy: {max_accuracy_gbc} (from {best_accuracy_gbc_model})")

# Store the variable name of the best model
%store best_accuracy_gbc_model
%store max_accuracy_gbc

Max accuracy: 0.921875 (from GradientBoostingClassifier(learning_rate=1, max_depth=7, n_estimators=90,
                           random_state=42))
Stored 'best_accuracy_gbc_model' (GradientBoostingClassifier)
Stored 'max_accuracy_gbc' (float64)
